Модель RandomForestClassifier 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Завантаження даних
df = pd.read_csv(r'archive\internet_service_churn.csv')

# Попередня обробка даних (закодування категоріальних змінних)
df = pd.get_dummies(df, drop_first=True)

# Заповнюємо стовбець reamining_contract нульом
data_mode = df.copy()
data_mode['reamining_contract'] = data_mode['reamining_contract'].fillna(data_mode['reamining_contract'].mode()[0])


# Відділяємо ознаки і цільову змінну
X = data_mode.drop('churn', axis=1)  # Все крім стовпця 'Churn'
y = data_mode['churn']  # Цільова змінна

# Розділяємо дані на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормалізація даних (StandardScaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Вибір алгоритму (RandomForest)
model = RandomForestClassifier(random_state=42)

# Крос-валідація з RandomizedSearch для пошуку найкращих параметрів
param_grid = {
    'n_estimators': [100, 200],  # Зменшено кількість варіантів
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Використовуємо RandomizedSearchCV замість GridSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, 
                                   n_iter=10, cv=3, scoring='accuracy', n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)

# Найкращі параметри
best_model = random_search.best_estimator_
print(f'Найкращі параметри: {random_search.best_params_}')

# Прогнозування
y_pred = best_model.predict(X_test)

# Оцінювання моделі
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Виведення докладного звіту
print(classification_report(y_test, y_pred))

# Матриця неточностей
conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Матриця неточностей:\n{conf_matrix}')


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Найкращі параметри: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}
Accuracy: 0.9684
Precision: 0.9705
Recall: 0.9717
F1 Score: 0.9711
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      6553
           1       0.97      0.97      0.97      7902

    accuracy                           0.97     14455
   macro avg       0.97      0.97      0.97     14455
weighted avg       0.97      0.97      0.97     14455

Матриця неточностей:
[[6320  233]
 [ 224 7678]]
